In [2]:
pip install tensorflow


   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 1.0/331.9 MB 5.0 MB/s eta 0:01:06
   ---------------------------------------- 3.9/331.9 MB 8.7 MB/s eta 0:00:38
   ---------------------------------------- 3.9/331.9 MB 8.7 MB/s eta 0:00:38
    --------------------------------------- 5.2/331.9 MB 6.0 MB/s eta 0:00:55
    --------------------------------------- 5.2/331.9 MB 6.0 MB/s eta 0:00:55
    --------------------------------------- 5.2/331.9 MB 6.0 MB/s eta 0:00:55
    --------------------------------------- 5.5/331.9 MB 3.7 MB/s eta 0:01:29
    --------------------------------------- 6.6/331.9 MB 3.8 MB/s eta 0:01:26
    --------------------------------------- 7.9/331.9 MB 4.0 MB/s eta 0:01:21
   - -------------------------------------- 8.9/331.9 MB 4.2 MB/s eta 0:01:17
   - -------------------------------------- 9.4/331.9 MB 4.3 MB/s eta 0:01:16


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.32.1 which is incompatible.


In [9]:
import pandas as pd
import numpy as np

import optuna

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn.metrics import roc_auc_score

In [10]:
tf.__version__

'2.20.0'

In [11]:
data = pd.read_csv('Employee.csv')

data

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [12]:
data.describe(include='all')

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
count,4653,4653.000000,4653,4653.000000,4653.000000,4653,4653,4653.000000,4653.000000
unique,3,NaN,3,NaN,NaN,2,2,NaN,NaN
top,Bachelors,NaN,Bangalore,NaN,NaN,Male,No,NaN,NaN
freq,3601,NaN,2228,NaN,NaN,2778,4175,NaN,NaN
mean,NaN,2015.062970,NaN,2.698259,29.393295,NaN,NaN,2.905652,0.343864
std,NaN,1.863377,NaN,0.561435,4.826087,NaN,NaN,1.558240,0.475047
min,NaN,2012.000000,NaN,1.000000,22.000000,NaN,NaN,0.000000,0.000000
25%,NaN,2013.000000,NaN,3.000000,26.000000,NaN,NaN,2.000000,0.000000
50%,NaN,2015.000000,NaN,3.000000,28.000000,NaN,NaN,3.000000,0.000000
75%,NaN,2017.000000,NaN,3.000000,32.000000,NaN,NaN,4.000000,1.000000


In [13]:
data.isnull().sum()

Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
LeaveOrNot                   0
dtype: int64

In [14]:
data = pd.get_dummies(data, drop_first=True)

data

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2017,3,34,0,0,False,False,False,False,True,False
1,2013,1,28,3,1,False,False,False,True,False,False
2,2014,3,38,2,0,False,False,True,False,False,False
3,2016,3,27,5,1,True,False,False,False,True,False
4,2017,3,24,2,1,True,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
4648,2013,3,26,4,0,False,False,False,False,False,False
4649,2013,2,37,2,1,True,False,False,True,True,False
4650,2018,3,27,5,1,True,False,True,False,True,False
4651,2012,3,30,2,0,False,False,False,False,True,True


In [15]:
targets = data['LeaveOrNot']

inputs = data.drop(['LeaveOrNot'],axis=1)

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,1.039638,0.537503,0.954645,-1.864901,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
1,-1.107233,-3.025177,-0.288732,0.060554,-0.480575,-0.200022,-0.575282,1.633878,-1.217210,-0.338365
2,-0.570515,0.537503,1.783563,-0.581264,-0.480575,-0.200022,1.738277,-0.612041,-1.217210,-0.338365
3,0.502921,0.537503,-0.495961,1.344191,2.080840,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
4,1.039638,0.537503,-1.117650,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,2.955387
...,...,...,...,...,...,...,...,...,...,...
4648,-1.107233,0.537503,-0.703191,0.702373,-0.480575,-0.200022,-0.575282,-0.612041,-1.217210,-0.338365
4649,-1.107233,-1.243837,1.576334,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,-0.338365
4650,1.576356,0.537503,-0.495961,1.344191,2.080840,-0.200022,1.738277,-0.612041,0.821551,-0.338365
4651,-1.643951,0.537503,0.125727,-0.581264,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,2.955387


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

In [18]:
def create_model(trial):
    # Building artificial neural network
    model = Sequential()

     # we add 2 hidden layers and 1 output layer
    model.add(Dense(units=trial.suggest_int('units_layer1', 6, 32), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 6, 32), activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    # Suggest hyperparameters for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    
    return model

In [19]:
def optimal(trial):
    
    # Suggest the number of epochs and batch size
    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    model = create_model(trial)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-09-27 12:22:43,221] A new study created in memory with name: no-name-42d626ce-d30f-4e2b-9bdc-2a05f10a47f0
C:\Users\user\AppData\Local\Temp\ipykernel_18584\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - AUC: 0.5310 - loss: 0.6883
Epoch 2/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5532 - loss: 0.6800
Epoch 3/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.5718 - loss: 0.6724
Epoch 4/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5908 - loss: 0.6651
Epoch 5/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6054 - loss: 0.6582
Epoch 6/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6190 - loss: 0.6519
Epoch 7/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6308 - loss: 0.6458 
Epoch 8/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6415 - loss: 0.6401
Epoch 9/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6519 - loss: 0.6345
Epoch 10/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6612 - loss: 0.6293
Epoch 11/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6691 - loss: 0.6244
Epoch 12/14
113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.6769 - loss: 0.6198
Epoch 13/14
113/113 ━━━━

[I 2025-09-27 12:22:48,565] Trial 0 finished with value: 0.724901690414177 and parameters: {'epochs': 14, 'batch_size': 33, 'units_layer1': 28, 'units_layer2': 9, 'optimizer': 'adam', 'learning_rate': 5.013988358641449e-05}. Best is trial 0 with value: 0.724901690414177.


Epoch 1/27


C:\Users\user\AppData\Local\Temp\ipykernel_18584\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


...

Epoch 1/20


C:\Users\user\AppData\Local\Temp\ipykernel_18584\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7571 - loss: 0.5387
Epoch 2/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8167 - loss: 0.4687
Epoch 3/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8371 - loss: 0.4374
Epoch 4/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8510 - loss: 0.4149
Epoch 5/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8571 - loss: 0.4006
Epoch 6/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8617 - loss: 0.3912
Epoch 7/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8657 - loss: 0.3851
Epoch 8/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8683 - loss: 0.3789
Epoch 9/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8711 - loss: 0.3756
Epoch 10/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8742 - loss: 0.3731
Epoch 11/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8712 - loss: 0.3725
Epoch 12/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8734 - loss: 0.3701
Epoch 13/20
233/233 ━━━━━━━━━━━━━━━━

[I 2025-09-27 12:23:55,780] Trial 9 finished with value: 0.8928374444614677 and parameters: {'epochs': 20, 'batch_size': 16, 'units_layer1': 16, 'units_layer2': 29, 'optimizer': 'rmsprop', 'learning_rate': 0.004393941727938682}. Best is trial 9 with value: 0.8928374444614677.


Best trial: 0.8928374444614677
Best hyperparameters: {'epochs': 20, 'batch_size': 16, 'units_layer1': 16, 'units_layer2': 29, 'optimizer': 'rmsprop', 'learning_rate': 0.004393941727938682}


In [20]:
best_params = study.best_trial.params

best_params

{'epochs': 20,
 'batch_size': 16,
 'units_layer1': 16,
 'units_layer2': 29,
 'optimizer': 'rmsprop',
 'learning_rate': 0.004393941727938682}

In [21]:
# Train the final model with the best hyperparameters

best_model = Sequential()
best_model.add(Dense(units=best_params['units_layer1'], activation='relu'))
best_model.add(Dense(units=best_params['units_layer2'], activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

In [22]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])


In [23]:
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['AUC'])

In [24]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'])
    
    '''Predictions and probabilities for the training set'''
    
    y_train_prob = model.predict(X_train)

    '''Predictions and probabilities for the test set'''
    
    y_test_prob = model.predict(X_test)

    '''Calculate metrics for the training set''' 
    
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1
    

    '''Calculate metrics for the test set'''
    
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1
    

    results = pd.DataFrame({
        'Dataset': ['Train', 'Test'],
        'Gini': [gini_train_prob * 100, gini_test_prob * 100],
    
    })

    return results

In [25]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.7604 - loss: 0.5371
Epoch 2/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8199 - loss: 0.4673
Epoch 3/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8407 - loss: 0.4287
Epoch 4/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8534 - loss: 0.4100
Epoch 5/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8608 - loss: 0.3969
Epoch 6/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8604 - loss: 0.3919
Epoch 7/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8639 - loss: 0.3849
Epoch 8/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8658 - loss: 0.3808
Epoch 9/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8723 - loss: 0.3786
Epoch 10/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8708 - loss: 0.3747
Epoch 11/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8697 - loss: 0.3750
Epoch 12/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8707 - loss: 0.3753
Epoch 13/20
233/233 ━━━━━

,Dataset,Gini
0,Train,77.032814
1,Test,78.316736


In [64]:

data_deploy = {
    'Education': ['Bachelors', 'Bachelors', 'Bachelors', 'Masters', 'Masters',
                  'PHD', 'Bachelors', 'Masters', 'Masters', 'Bachelors'],
    'JoiningYear': [2017, 2013, 2014, 2016, 2017,
                    2012, 2015, 2018, 2011, 2019],
    'City': ['Bangalore', 'Pune', 'New Delhi', 'Bangalore', 'Pune',
             'New Delhi', 'Bangalore', 'Pune', 'Bangalore', 'New Delhi'],
    'PaymentTier': [3, 1, 3, 3, 3,
                    2, 3, 3, 1, 2],
    'Age': [34, 28, 38, 27, 24,
            41, 29, 26, 35, 32],
    'Gender': ['Male', 'Female', 'Female', 'Male', 'Male',
               'Female', 'Male', 'Male', 'Female', 'Male'],
    'EverBenched': ['No', 'No', 'No', 'No', 'Yes',
                    'No', 'Yes', 'No', 'No', 'Yes'],
    'ExperienceInCurrentDomain': [0, 3, 2, 5, 2,
                                  6, 4, 1, 7, 2]
}

employee_deploy = pd.DataFrame(data_deploy)
employee_deploy


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,Bachelors,2017,Bangalore,3,34,Male,No,0
1,Bachelors,2013,Pune,1,28,Female,No,3
2,Bachelors,2014,New Delhi,3,38,Female,No,2
3,Masters,2016,Bangalore,3,27,Male,No,5
4,Masters,2017,Pune,3,24,Male,Yes,2
5,PHD,2012,New Delhi,2,41,Female,No,6
6,Bachelors,2015,Bangalore,3,29,Male,Yes,4
7,Masters,2018,Pune,3,26,Male,No,1
8,Masters,2011,Bangalore,1,35,Female,No,7
9,Bachelors,2019,New Delhi,2,32,Male,Yes,2


In [65]:
employee_deploy.describe(include='all')

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
count,10,10.00000,10,10.000000,10.000000,10,10,10.000000
unique,3,NaN,3,NaN,NaN,2,2,NaN
top,Bachelors,NaN,Bangalore,NaN,NaN,Male,No,NaN
freq,5,NaN,4,NaN,NaN,6,7,NaN
mean,NaN,2015.20000,NaN,2.400000,31.400000,NaN,NaN,3.200000
std,NaN,2.65832,NaN,0.843274,5.541761,NaN,NaN,2.250926
min,NaN,2011.00000,NaN,1.000000,24.000000,NaN,NaN,0.000000
25%,NaN,2013.25000,NaN,2.000000,27.250000,NaN,NaN,2.000000
50%,NaN,2015.50000,NaN,3.000000,30.500000,NaN,NaN,2.500000
75%,NaN,2017.00000,NaN,3.000000,34.750000,NaN,NaN,4.750000


In [66]:
employee_deploy.isnull().sum()

Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
dtype: int64

In [67]:
employee_deploy1 = pd.get_dummies(employee_deploy,dtype=int)

employee_deploy1

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Bachelors,Education_Masters,Education_PHD,City_Bangalore,City_New Delhi,City_Pune,Gender_Female,Gender_Male,EverBenched_No,EverBenched_Yes
0,2017,3,34,0,1,0,0,1,0,0,0,1,1,0
1,2013,1,28,3,1,0,0,0,0,1,1,0,1,0
2,2014,3,38,2,1,0,0,0,1,0,1,0,1,0
3,2016,3,27,5,0,1,0,1,0,0,0,1,1,0
4,2017,3,24,2,0,1,0,0,0,1,0,1,0,1
5,2012,2,41,6,0,0,1,0,1,0,1,0,1,0
6,2015,3,29,4,1,0,0,1,0,0,0,1,0,1
7,2018,3,26,1,0,1,0,0,0,1,0,1,1,0
8,2011,1,35,7,0,1,0,1,0,0,1,0,1,0
9,2019,2,32,2,1,0,0,0,1,0,0,1,0,1


In [68]:
scaler.fit(employee_deploy1)

scaled = scaler.transform(employee_deploy1)

df_test_scaled = pd.DataFrame(scaled, columns=employee_deploy1.columns)

df_test_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Bachelors,Education_Masters,Education_PHD,City_Bangalore,City_New Delhi,City_Pune,Gender_Female,Gender_Male,EverBenched_No,EverBenched_Yes
0,0.713746,0.75,0.494543,-1.498537,1.0,-0.816497,-0.333333,1.224745,-0.654654,-0.654654,-0.816497,0.816497,0.654654,-0.654654
1,-0.872357,-1.75,-0.646710,-0.093659,1.0,-0.816497,-0.333333,-0.816497,-0.654654,1.527525,1.224745,-1.224745,0.654654,-0.654654
2,-0.475831,0.75,1.255379,-0.561951,1.0,-0.816497,-0.333333,-0.816497,1.527525,-0.654654,1.224745,-1.224745,0.654654,-0.654654
3,0.317221,0.75,-0.836919,0.842927,-1.0,1.224745,-0.333333,1.224745,-0.654654,-0.654654,-0.816497,0.816497,0.654654,-0.654654
4,0.713746,0.75,-1.407546,-0.561951,-1.0,1.224745,-0.333333,-0.816497,-0.654654,1.527525,-0.816497,0.816497,-1.527525,1.527525
5,-1.268883,-0.50,1.826006,1.311220,-1.0,-0.816497,3.000000,-0.816497,1.527525,-0.654654,1.224745,-1.224745,0.654654,-0.654654
6,-0.079305,0.75,-0.456502,0.374634,1.0,-0.816497,-0.333333,1.224745,-0.654654,-0.654654,-0.816497,0.816497,-1.527525,1.527525
7,1.110272,0.75,-1.027128,-1.030244,-1.0,1.224745,-0.333333,-0.816497,-0.654654,1.527525,-0.816497,0.816497,0.654654,-0.654654
8,-1.665408,-1.75,0.684752,1.779513,-1.0,1.224745,-0.333333,1.224745,-0.654654,-0.654654,1.224745,-1.224745,0.654654,-0.654654
9,1.506798,-0.50,0.114125,-0.561951,1.0,-0.816497,-0.333333,-0.816497,1.527525,-0.654654,-0.816497,0.816497,-1.527525,1.527525


In [69]:
X_train.columns

Index(['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain',
       'Education_Masters', 'Education_PHD', 'City_New Delhi', 'City_Pune',
       'Gender_Male', 'EverBenched_Yes'],
      dtype='object')

In [70]:
df_test_scaled= df_test_scaled[['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain',
       'Education_Masters', 'Education_PHD', 'City_New Delhi', 'City_Pune',
       'Gender_Male', 'EverBenched_Yes']]

In [71]:
employee_deploy['predicted_LeaveOrNot'] = best_model.predict(df_test_scaled)


employee_deploy

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,predicted_LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0.153726
1,Bachelors,2013,Pune,1,28,Female,No,3,0.999983
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0.076762
3,Masters,2016,Bangalore,3,27,Male,No,5,0.820474
4,Masters,2017,Pune,3,24,Male,Yes,2,0.515983
5,PHD,2012,New Delhi,2,41,Female,No,6,0.163099
6,Bachelors,2015,Bangalore,3,29,Male,Yes,4,0.152826
7,Masters,2018,Pune,3,26,Male,No,1,0.980184
8,Masters,2011,Bangalore,1,35,Female,No,7,0.972061
9,Bachelors,2019,New Delhi,2,32,Male,Yes,2,0.555262
